# Форматы данных (2)

Материалы:
* Макрушин С.В. "Лекция 5: Форматы данных (часть 2)"
* https://docs.python.org/3/library/csv.html
* https://docs.h5py.org/en/stable/
* Уэс Маккини. Python и анализ данных

In [25]:
import csv
import json
import pprint
from collections import defaultdict
from timeit import default_timer

import h5py
import numpy as np
import pandas as pd

In [ ]:
def pprint_dict_of_lists(dct: dict, n=3, m=10):
    pprint.pprint({k: [*v[:m], '...'] for k, v in list(dct.items())[:n]}, sort_dicts=False)

In [3]:
## Задачи для совместного разбора

1. Считайте данные из файла `open_pubs.csv`, используя `csv.reader`, и преобразуйте к структуре данных следующего вида:
    
`{'fas_id': [24, 30, ...], 'name': ['Achor Inn', 'Angel Inn', ...], ... }`


In [4]:
with open('data/open_pubs.csv') as f:
    reader = csv.reader(f)

    headers = next(reader)
    n = len(headers)
    data = {header: [] for header in headers}

    for row in reader:
        for i in range(n):
            data[headers[i]].append(row[i])

pprint_dict_of_lists(data, n=len(data), m=6)

{'fas_id': ['24', '30', '63', '64', '65', '85', '...'],
 'name': ['Anchor Inn',
          'Angel Inn',
          'Black Boy Hotel',
          'Black Horse',
          'Black Lion',
          'Bristol Arms',
          '...'],
 'address': ['Upper Street, Stratford St Mary, COLCHESTER, Essex',
             'Egremont Street, Glemsford, SUDBURY, Suffolk',
             '7 Market Hill, SUDBURY, Suffolk',
             'Lower Street, Stratford St Mary, COLCHESTER, Essex',
             'Lion Road, Glemsford, SUDBURY, Suffolk',
             'Bristol Hill, Shotley, IPSWICH, Suffolk',
             '...'],
 'postcode': ['CO7 6LW',
              'CO10 7SA',
              'CO10 2EA',
              'CO7 6JS',
              'CO10 7RF',
              'IP9 1PU',
              '...'],
 'easting': ['604748', '582888', '587356', '604270', '582750', '624667', '...'],
 'northing': ['234405',
              '247368',
              '241327',
              '233920',
              '248298',
              '233744',


2. Сгенерируйте 2 случайные матрицы размера 10_000 x 10_000 и вычислите их произведение. Сколько времени занимают три этих операции? Сохраните 3 полученных матрицы в файл .npz с соответствующими названиями

In [7]:
start = default_timer()

size = (3000, 3000)
matrix_0 = np.random.randint(0, 100, size=size, dtype=np.int8)
matrix_1 = np.random.randint(0, 100, size=size, dtype=np.int8)

print(f'time: {(default_timer() - start) * 1000:.4f} ms')

time: 149.7895 ms


In [8]:
start = default_timer()

matrix_dot = np.dot(matrix_0, matrix_1)

print(f'time: {(default_timer() - start) * 1000:.4f} ms')

time: 76557.9985 ms


In [9]:
print(matrix_0)
print(matrix_1)
print(matrix_dot)

[[66  7 65 ... 15 11 25]
 [84 40 83 ... 76  2 85]
 [28 30 42 ... 83 88 31]
 ...
 [15 16 91 ... 78 42 78]
 [39 56 25 ... 86 67 15]
 [14 81 67 ... 31 67 89]]
[[40 25 62 ... 68 25 46]
 [64 18  5 ... 87 92 57]
 [24 24 32 ... 33 22 98]
 ...
 [13 92 22 ... 81 74 63]
 [72 37 10 ... 67  1 11]
 [59 23 74 ... 18 67 53]]
[[ -54  -93 -117 ...  -71 -102    4]
 [  -1  -63  -62 ...  -58   44  -49]
 [-127  -18 -103 ...   31  116  -76]
 ...
 [ -96  104  -30 ...    2   62   11]
 [  92 -121   66 ...  -31  121  -58]
 [ -52  -50  -46 ...  -85   99 -102]]


In [10]:
np.savez('data/output/matrices.npz', matrix_0=matrix_0, matrix_1=matrix_1, matrix_dot=matrix_dot)

3. Создайте 2 матрицы размера 1000x1000, используя различные параметризируемые распределения из numpy (https://docs.scipy.org/doc/numpy-1.15.0/reference/routines.random.html#distributions)

После этого сохраните получившиеся матрицы в hdf5-файл в виде двух различных датасетов. В качестве описания каждого датасета укажите параметры используемых распределений 

In [11]:
size = (1000, 1000)
matrix_0 = np.random.uniform(low=-20, high=50, size=size)
matrix_1 = np.random.normal(loc=10, scale=10, size=size)

In [12]:
with h5py.File('data/output/matrices.hdf5', mode='w') as f:
    dset_0 = f.create_dataset('matrix_0', data=matrix_0)
    dset_0.attrs['description'] = ('Матрица, сгенерированная с помощью np.random.uniform(). '
                                   '(Draw samples from a uniform distribution)')
    dset_0.attrs['low'] = '-20'
    dset_0.attrs['high'] = '50'
    dset_0.attrs['size'] = f'{matrix_0.shape}'

    dset_1 = f.create_dataset('matrix_1', data=matrix_1)
    dset_1.attrs['description'] = ('Матрица, сгенерированная с помощью np.random.normal(). '
                                   '(Draw random samples from a normal (Gaussian) distribution)')
    dset_1.attrs['loc'] = '10'
    dset_1.attrs['scale'] = '10'
    dset_1.attrs['size'] = f'{matrix_1.shape}'

In [13]:
with h5py.File('data/output/matrices.hdf5') as f:
    for key in f.keys():
        dset_1 = f[key]
        print(dset_1.attrs['description'])
        print(dset_1[:].shape)

Матрица, сгенерированная с помощью np.random.uniform(). (Draw samples from a uniform distribution)
(1000, 1000)
Матрица, сгенерированная с помощью np.random.normal(). (Draw random samples from a normal (Gaussian) distribution)
(1000, 1000)


## Лабораторная работа 5

### csv

1.1 В файле `tags_sample.csv` находится информация о тэгах, приписываемых рецептам. Воспользовавшись `csv.reader`, считайте этот файл и создайте словарь вида `id_рецепта: [список тэгов]`. Сохраните этот словарь в файл `tags_sample.json`.

In [52]:
with open('data/tags_sample.csv') as f:
    reader = csv.reader(f)

    headers = next(reader)
    n = len(headers)

    tags_sample = defaultdict(list)
    for row in reader:
        tags_sample[int(row[0])].append(row[1])

In [53]:
with open('data/output/tags_sample.json', mode='w', encoding='utf-8') as f:
    json.dump(tags_sample, f, indent=4)

1.2 Считайте файл `recipes_sample_with_filled_nsteps.csv` (__ЛР4__) в виде `pd.DataFrame`. Добавьте к таблице 2 столбца: `n_tags`, содержащий количество тэгов у этого рецепта; и `tags`, содержащий набор тэгов в виде строки (тэги внутри строки разделяются символом `;`)

In [54]:
recipes_df = pd.read_csv(
    'data/recipes_sample_with_filled_nsteps.csv',
    sep=',', index_col=0, parse_dates=['submitted']
)

In [55]:
recipes_df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,7,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4,i think a fondue is a very romantic casual din...,NaN
...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16,this is based on a french recipe but i changed...,10.0
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22,"this is a traditional fresh plum cake, thought...",11.0
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10,this is a traditional late summer early fall s...,NaN
29998,zydeco soup,486161,60,227978,2012-08-29,7,this is a delicious soup that i originally fou...,NaN


In [56]:
n_tags_tags = {'id': [], 'n_tags': [], 'tags': []}
for k, v in tags_sample.items():
    n_tags_tags['id'].append(k)
    n_tags_tags['n_tags'].append(len(v))
    n_tags_tags['tags'].append(';'.join(v))

n_tags_tags_df = pd.DataFrame(n_tags_tags)

In [57]:
n_tags_tags_df

,id,n_tags,tags
0,44123,25,weeknight;time-to-make;course;main-ingredient;...
1,67664,31,15-minutes-or-less;time-to-make;course;prepara...
2,38798,17,30-minutes-or-less;time-to-make;course;main-in...
3,35173,11,60-minutes-or-less;time-to-make;course;prepara...
4,84797,19,30-minutes-or-less;time-to-make;course;main-in...
...,...,...,...
29995,267661,18,time-to-make;course;main-ingredient;cuisine;pr...
29996,386977,19,time-to-make;course;main-ingredient;cuisine;pr...
29997,103312,20,time-to-make;course;main-ingredient;cuisine;pr...
29998,486161,20,ham;60-minutes-or-less;time-to-make;course;mai...


In [59]:
recipes_tags_df = pd.merge(
    recipes_df,
    n_tags_tags_df,
    how='left',
    on='id'
)

In [60]:
recipes_tags_df

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients,n_tags,tags
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,11,an original recipe created by chef scott meska...,18.0,25,weeknight;time-to-make;course;main-ingredient;...
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,3,my children and their friends ask for my homem...,NaN,31,15-minutes-or-less;time-to-make;course;prepara...
2,i can t believe it s spinach,38798,30,1533,2002-08-29,5,"these were so go, it surprised even me.",8.0,17,30-minutes-or-less;time-to-make;course;main-in...
3,italian gut busters,35173,45,22724,2002-07-27,7,my sister-in-law made these for us at a family...,NaN,11,60-minutes-or-less;time-to-make;course;prepara...
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4,i think a fondue is a very romantic casual din...,NaN,19,30-minutes-or-less;time-to-make;course;main-in...
...,...,...,...,...,...,...,...,...,...,...
29995,zurie s holey rustic olive and cheddar bread,267661,80,200862,2007-11-25,16,this is based on a french recipe but i changed...,10.0,18,time-to-make;course;main-ingredient;cuisine;pr...
29996,zwetschgenkuchen bavarian plum cake,386977,240,177443,2009-08-24,22,"this is a traditional fresh plum cake, thought...",11.0,19,time-to-make;course;main-ingredient;cuisine;pr...
29997,zwiebelkuchen southwest german onion cake,103312,75,161745,2004-11-03,10,this is a traditional late summer early fall s...,NaN,20,time-to-make;course;main-ingredient;cuisine;pr...
29998,zydeco soup,486161,60,227978,2012-08-29,7,this is a delicious soup that i originally fou...,NaN,20,ham;60-minutes-or-less;time-to-make;course;mai...


1.3 В файле `ingredients_sample.csv` находится информация о ингредиентах, необходимых для рецепта. Воспользовавшись `csv.DictReader`, считайте этот файл и создайте словарь вида `id_рецепта: [список ингредиентов]`.

1.4 Добавьте к таблице из задания 1.2 столбец `ingredients`, содержащий набор ингредиентов в виде строки (ингредиенты внутри строки разделяются символом `*`)

Для строк, которые содержат пропуски в столбце `n_ingredients`, заполните их на основе файла  `ingredients_sample.csv`

1.5 Проверьте, содержит ли столбец `n_ingredients` пропуски. Если нет, треобразуйте его к целочисленному типу и сохраните результаты в файл `recipes_sample_with_tags_ingredients.csv`

### npy

2.1 Разделите таблицу, полученную в результате 1.5, на две таблицы: одна содержит рецепты, загруженные до 2000 года; вторая - все остальные. В полученных таблицах оставьте только числовые столбцы и преобразуйте их к `numpy.array`

2.2. Сохраните 2 полученных массива в архив `npz`. Дайте массивам читаемые имена.

2.3 Считайте созданный архив и продемонстрируйте, что данные считались корректно. 

### hdf

3.1 Выведите названия всех датасетов, находящихся в файле `nutrition_sample.h5`, а также размерность матриц, содержащихся в данных датасетах и их метаданные.

Формат вывода:
```
Dataset name=dataset_0, dataset size=(30000,), metadata={'info': 'calories (#)'}
Dataset name=dataset_1, dataset size=(30000,), metadata={'info': 'total fat (PDV)'}
...
```

3.2 Разбейте каждый из имеющихся датасетов на две части: 1 часть содержит только те строки, где PDV (Percent Daily Value) превышает 100%; 2 часть содержит те строки, где PDV не составляет не более 100%. Создайте 2 группы в файле и разместите в них соответствующие части датасета c сохранением метаданных исходных датасетов. Итого должно получиться 2 группы, содержащие несколько датасетов. Сохраните результаты в файл `nutrition_grouped.h5`

3.3 Выведите названия всех групп и датасетов, находящихся в этих группах, из файла `nutrition_grouped.h5` а также размерность матриц, содержащихся в датасетах и их метаданные.

3.4 Модифицируйте код из 3.3 таким образом, чтобы сохранить датасеты, используя сжатие. Сравните размер полученного файла с размерами файла из 3.3. Прокомментируйте результат.